In [1]:
import datetime
import json
import pandas as pd

# Define a dictionary to map sleep stage values to their corresponding labels
sleep_stage_labels = {
    1: "Awake (during sleep cycle)",
    2: "Sleep",
    3: "Out-of-bed",
    4: "Light sleep",
    5: "Deep sleep",
    6: "REM"
}

# Read sleep segment data from file
data = pd.read_json("..\Fit\All Data\derived_com.google.sleep.segment_com.google.an.json")
sleep_segments = data["Data Points"]

# Convert Epoch time to datetime objects and adjust dates
for segment in sleep_segments:
    start_time = datetime.datetime.fromtimestamp(int(segment["startTimeNanos"]) / 1e9)
    end_time = datetime.datetime.fromtimestamp(int(segment["endTimeNanos"]) / 1e9)
    if end_time.hour >= 16:
        segment_date = end_time.date()
    else:
        segment_date = start_time.date()
    segment["start_time"] = start_time
    segment["end_time"] = end_time
    segment["date"] = segment_date

# Calculate duration of each sleep segment in hours
for segment in sleep_segments:
    duration = (segment["end_time"] - segment["start_time"]).total_seconds() / 3600
    segment["duration"] = duration

# Group sleep segments by date and sum durations for each sleep stage
totals = {}
for segment in sleep_segments:
    date = segment["date"]
    sleep_stage = segment["fitValue"][0]["value"]["intVal"]
    if date not in totals:
        totals[date] = {label: 0 for label in sleep_stage_labels.values()}
    totals[date][sleep_stage_labels[sleep_stage]] += segment["duration"]

# Create a pandas DataFrame with date and hours slept for each sleep stage
df = pd.DataFrame.from_dict(totals, orient='index')
df.index.name = 'Date'
df.reset_index(inplace=True)

# Add a Total column to the DataFrame
df["Total"] = df.iloc[:, 1:].sum(axis=1)

# Calculate actual sleep time by subtracting "Awake" duration from "Total" duration
df["Actual Sleep Time"] = df["Total"] - df["Awake (during sleep cycle)"]

# Move "Actual Sleep Time" column to the left of "Total" column
df = df[["Date", "Awake (during sleep cycle)", "Sleep", "Out-of-bed", "Light sleep", "Deep sleep", "REM", "Actual Sleep Time", "Total"]]

# Sort the dataframe by date
df = df.sort_values(by="Date")

print(df)

           Date  Awake (during sleep cycle)  Sleep  Out-of-bed  Light sleep   
537  2019-07-28                    0.066667    0.0           0     0.958333  \
538  2019-07-29                    0.983333    0.0           0     3.691667   
539  2019-07-30                    0.783333    0.0           0     5.158333   
540  2019-07-31                    0.566667    0.0           0     3.675000   
541  2019-08-01                    0.008333    0.0           0     0.991667   
..          ...                         ...    ...         ...          ...   
532  2023-04-27                    0.716667    0.0           0     4.133333   
533  2023-04-28                    0.900000    0.0           0     5.366667   
534  2023-04-29                    1.133333    0.0           0     6.650000   
535  2023-04-30                    0.049999    0.0           0     0.916665   
536  2023-05-01                    0.533327    0.0           0     4.566658   

     Deep sleep       REM  Actual Sleep Time     To

In [2]:
df

,Date,Awake (during sleep cycle),Sleep,Out-of-bed,Light sleep,Deep sleep,REM,Actual Sleep Time,Total
537,2019-07-28,0.066667,0.0,0,0.958333,0.083333,0.000000,1.041667,1.108333
538,2019-07-29,0.983333,0.0,0,3.691667,1.416667,1.925000,7.033333,8.016667
539,2019-07-30,0.783333,0.0,0,5.158333,1.933333,2.058333,9.150000,9.933333
540,2019-07-31,0.566667,0.0,0,3.675000,0.466667,2.175000,6.316667,6.883333
541,2019-08-01,0.008333,0.0,0,0.991667,0.641667,0.158333,1.791667,1.800000
...,...,...,...,...,...,...,...,...,...
532,2023-04-27,0.716667,0.0,0,4.133333,0.000000,2.516667,6.650000,7.366666
533,2023-04-28,0.900000,0.0,0,5.366667,0.700000,1.966667,8.033334,8.933334
534,2023-04-29,1.133333,0.0,0,6.650000,0.416667,1.550000,8.616667,9.750000
535,2023-04-30,0.049999,0.0,0,0.916665,0.566666,0.033333,1.516664,1.566663
